In [1]:
from thesis_v2.analysis.utils import LayerSourceAnalysis, get_source_analysis_for_one_model_spec

In [2]:
# check results in schema.pdf that explains multi path idea

def check_schema_figure():
    for config in [(1,3), (2,2)]:
        num_l, num_t = config
        aaaaaa = get_source_analysis_for_one_model_spec(
            num_recurrent_layer=num_l, num_cls=num_t,
            readout_type='inst-last', return_raw=True, add_bn_in_chain=True
        )[-1]
        for zzz in aaaaaa:
            print(zzz.source_list)
                
check_schema_figure()

[{'conv': ('I', 'B1', 's1,1'), 'scale': (1.0,)}]
[{'conv': ('I', 'B1', 's1,2'), 'scale': (1.0,)}, {'conv': ('I', 'B1', 's1,1', 'R1', 's1,2'), 'scale': (1.0,)}]
[{'conv': ('I', 'B1', 's1,3'), 'scale': (1.0,)}, {'conv': ('I', 'B1', 's1,2', 'R1', 's1,3'), 'scale': (1.0,)}, {'conv': ('I', 'B1', 's1,1', 'R1', 's1,2', 'R1', 's1,3'), 'scale': (1.0,)}]
[{'conv': ('I', 'B1', 's1,1', 'B2', 's2,1'), 'scale': (1.0,)}]
[{'conv': ('I', 'B1', 's1,2', 'B2', 's2,2'), 'scale': (1.0,)}, {'conv': ('I', 'B1', 's1,1', 'R1', 's1,2', 'B2', 's2,2'), 'scale': (1.0,)}, {'conv': ('I', 'B1', 's1,1', 'B2', 's2,1', 'R2', 's2,2'), 'scale': (1.0,)}]


In [3]:
# use my util function to achieve the same

In [4]:
### let's see different readout types

def check_different_readout_type():
    scale_map = {
        # a default dict won't work!!! because .get() in defaultdict is the same is regular dict.
        f's1,{t}': 1.0 for t in range(1, 3+1)
    }
    
    for readout_type in ('inst-last', 'cm-last', 'inst-avg', 'cm-avg'):
        src_this = get_source_analysis_for_one_model_spec(
            num_recurrent_layer=1, num_cls=3, readout_type=readout_type
        )
        print(readout_type)
        print(src_this.unique_convs)
        print(src_this.source_list)
        print(src_this.evaluate(scale_map))

In [5]:
check_different_readout_type()
# looks good!

inst-last
{('I', 'B1', 'R1', 'R1'), ('I', 'B1', 'R1'), ('I', 'B1')}
[{'conv': ('I', 'B1'), 'scale': (1.0, 's1,3')}, {'conv': ('I', 'B1', 'R1'), 'scale': (1.0, 's1,2', 's1,3')}, {'conv': ('I', 'B1', 'R1', 'R1'), 'scale': (1.0, 's1,1', 's1,2', 's1,3')}]
{('I', 'B1'): 1.0, ('I', 'B1', 'R1'): 1.0, ('I', 'B1', 'R1', 'R1'): 1.0}
cm-last
{('I', 'B1', 'R1', 'R1'), ('I', 'B1', 'R1'), ('I', 'B1')}
[{'conv': ('I', 'B1'), 'scale': (1.0, 's1,1', 0.3333333333333333)}, {'conv': ('I', 'B1'), 'scale': (1.0, 's1,2', 0.3333333333333333)}, {'conv': ('I', 'B1', 'R1'), 'scale': (1.0, 's1,1', 's1,2', 0.3333333333333333)}, {'conv': ('I', 'B1'), 'scale': (1.0, 's1,3', 0.3333333333333333)}, {'conv': ('I', 'B1', 'R1'), 'scale': (1.0, 's1,2', 's1,3', 0.3333333333333333)}, {'conv': ('I', 'B1', 'R1', 'R1'), 'scale': (1.0, 's1,1', 's1,2', 's1,3', 0.3333333333333333)}]
{('I', 'B1'): 1.0, ('I', 'B1', 'R1'): 0.6666666666666666, ('I', 'B1', 'R1', 'R1'): 0.3333333333333333}
inst-avg
{('I', 'B1', 'R1', 'R1'), ('I', 'B1', 

In [6]:
### let's see different readout types

def check_different_readout_type_2layer():
    scale_map = {
        # a default dict won't work!!! because .get() in defaultdict is the same is regular dict.
        f's1,{t}': 1.0 for t in range(1, 5+1)
    }
    
    scale_map.update({
        f's2,{t}': 1.0 for t in range(1, 5+1)
    })
    
    scale_map.update({
        f's3,{t}': 1.0 for t in range(1, 5+1)
    })
    
    scale_map.update({
        f's4,{t}': 1.0 for t in range(1, 5+1)
    })
    for num_recurrent_layer, num_cls in [(1,1),(2,1),(3,1),(1,2),(1,3),(2,2),(3,4)]:
        for readout_type in ('inst-last', 'cm-last', 'inst-avg', 'cm-avg'):
            src_this = get_source_analysis_for_one_model_spec(
                num_recurrent_layer=num_recurrent_layer, num_cls=num_cls, readout_type=readout_type
                # 2,4 max length: I+5
                # 1,4 max length: I+4
                # 2,5 max length: I+6
                # 3,5 max length: I+7
                # 4,5 max length: I+8

                # adding one layer will only extend max length by 1.

                # (x,y) max length = I+y+(x-1)
            )
            print(readout_type)
            print(src_this.unique_convs)
            print(src_this.source_list)
            print('max length', max(len(x) for x in src_this.unique_convs))
            assert max(len(x) for x in src_this.unique_convs) == num_recurrent_layer + num_cls
            print(src_this.evaluate(scale_map))

In [7]:
check_different_readout_type_2layer()
# verified some of them. looks correct. it's so complicated.
# note that we can have multiple instances with same depth, such as 
# ('I', 'B1', 'R1', 'B2') vs ('I', 'B1', 'B2', 'R2').
# it will be try to analyze this.

inst-last
{('I', 'B1')}
[{'conv': ('I', 'B1'), 'scale': (1.0, 's1,1')}]
max length 2
{('I', 'B1'): 1.0}
cm-last
{('I', 'B1')}
[{'conv': ('I', 'B1'), 'scale': (1.0, 's1,1', 1.0)}]
max length 2
{('I', 'B1'): 1.0}
inst-avg
{('I', 'B1')}
[{'conv': ('I', 'B1'), 'scale': (1.0, 's1,1', 1.0)}]
max length 2
{('I', 'B1'): 1.0}
cm-avg
{('I', 'B1')}
[{'conv': ('I', 'B1'), 'scale': (1.0, 's1,1', 1.0, 1.0)}]
max length 2
{('I', 'B1'): 1.0}
inst-last
{('I', 'B1', 'B2')}
[{'conv': ('I', 'B1', 'B2'), 'scale': (1.0, 's1,1', 's2,1')}]
max length 3
{('I', 'B1', 'B2'): 1.0}
cm-last
{('I', 'B1', 'B2')}
[{'conv': ('I', 'B1', 'B2'), 'scale': (1.0, 's1,1', 's2,1', 1.0)}]
max length 3
{('I', 'B1', 'B2'): 1.0}
inst-avg
{('I', 'B1', 'B2')}
[{'conv': ('I', 'B1', 'B2'), 'scale': (1.0, 's1,1', 's2,1', 1.0)}]
max length 3
{('I', 'B1', 'B2'): 1.0}
cm-avg
{('I', 'B1', 'B2')}
[{'conv': ('I', 'B1', 'B2'), 'scale': (1.0, 's1,1', 's2,1', 1.0, 1.0)}]
max length 3
{('I', 'B1', 'B2'): 1.0}
inst-last
{('I', 'B1', 'B2', 'B3')}
